In [1]:
import pandas as pd
import csv

In [2]:
data = pd.read_csv('../../data/data1.csv',sep=';')
data

,ts,Consumo,Sensor 1,Sensor 2,potencia 1,potencia 2,voltaje
0,2021-07-06T05:00:21.779Z,32.345627,16.006992,16.338636,2017.132202,2058.924561,126.015694
1,2021-07-06T05:01:21.784Z,32.261688,16.060499,16.201187,2025.656250,2043.400635,126.126602
2,2021-07-06T05:02:21.782Z,33.683365,17.261465,16.421902,2176.436523,2070.578857,126.086426
3,2021-07-06T05:03:21.783Z,32.975597,16.544708,16.430889,2088.793213,2074.423340,126.251442
4,2021-07-06T05:04:21.785Z,38.917267,19.563198,19.354067,2466.679932,2440.311035,126.087761
...,...,...,...,...,...,...,...
38413,2021-08-06T04:55:35.810Z,12.799702,6.239447,6.560254,784.612549,824.954163,125.750336
38414,2021-08-06T04:56:35.815Z,12.842651,6.210476,6.632175,781.996704,835.095337,125.915749
38415,2021-08-06T04:57:35.811Z,12.731081,6.157272,6.573810,775.474243,827.934875,125.944458
38416,2021-08-06T04:58:35.821Z,12.989143,6.247071,6.742072,786.553345,848.877625,125.907539


In [3]:
p1_u = []
p2_u = []
consumo = []
vol_u = []
hora = []
fecha = []
dias = 0

watt_hora = 0
kw_hora = []
xhora = []

watt_dia = 0
kw_dia = []
xdia = []

lista = list(data['ts'])
dia_ini = int(str(lista[0])[8:10])
print(dia_ini)
hora_ini = int(str(lista[0])[11:13])
print(hora_ini)
j=0
ini = 0


6
5


In [4]:

for i in lista:
    u_p1 = float(data[data['ts'] == i]['potencia 1'].values)
    p1_u.append(u_p1)
    u_p2 = float(data[data['ts'] == i]['potencia 2'].values)
    p2_u.append(u_p2)
    vol_u = float(data[data['ts'] == i]['voltaje'].values)
    carga = u_p1+u_p2
    consumo.append(carga)
    tiempo = str(data[data['ts'] == i]['ts'].values)
    fecha.append(tiempo.split("T")[0][2:])
    hora.append(tiempo.split("T")[1][:8])
    watt_hora = watt_hora + carga/1000
    if j+1 < len(lista):
        # pasado una hora
        if hora_ini != int(str(lista[j+1])[11:13]):
            kw_hora.append(watt_hora)
            xhora.append(hora_ini)
            hora_ini = int(str(lista[j+1])[11:13])
            watt_dia = watt_dia + watt_hora
            watt_hora = 0
            print(i)
        # pasado un dia
        if dia_ini != int(str(lista[j+1])[8:10]) and int(str(lista[j+1])[11:13]) == 0:
            kw_dia.append(watt_dia)
            xdia.append(dia_ini)
            watt_dia = 0
            #ini = j + 1
            dias = dias + 1
            dia_ini = int(str(lista[j+1])[8:10])
        j = j+1

dias


2021-07-06T05:59:26.505Z
2021-07-06T06:59:21.967Z
2021-07-06T07:59:22.043Z
2021-07-06T08:59:22.137Z
2021-07-06T09:59:22.219Z
2021-07-06T10:58:25.550Z
2021-07-06T11:56:22.639Z
2021-07-06T12:59:22.481Z


KeyboardInterrupt: 

In [ ]:
data_final = pd.DataFrame({
    'hora' :hora,
    'fecha': fecha,
    'P1' :p1_u,
    'P2' :p2_u,
    'consumo': consumo
    })

In [ ]:
data_xhoras = pd.DataFrame({
    'hora' : xhora,
    'consumo' : kw_hora
})

In [ ]:
data_xdia = pd.DataFrame({
    'dia' : xdia,
    'consumo' : kw_dia
})

In [ ]:
data_final.to_csv('./final.csv')
data_xhoras.to_csv('./kw_hora.csv')
data_xdia.to_csv('./kw_dia.csv')